Import Libraries

In [1]:
import pandas as pd
import datetime
from datetime import datetime, timedelta
import os
import polars as pl
import gc
import numpy as np
pl.enable_string_cache() #disable_string_cache() for opposite effect.

import s3fs
import boto3
from io import BytesIO as bo

Calculate Date Variables

In [2]:
#variables -
this_day = datetime.today()
# ### FOR TESTING - REMOVE LATER ### 
# this_day = this_day - timedelta(days=7)
# ####
days_to_monday = (this_day.weekday() - 0) % 7
monday = this_day - timedelta(days=days_to_monday)

CUR_PROC_WK = monday.strftime("%Y%m%d")

PRE_PROC_WK0 = monday - timedelta(days=7)
PRE_PROC_WK = str(PRE_PROC_WK0.year) + str(PRE_PROC_WK0.month).zfill(2) + str(PRE_PROC_WK0.day).zfill(2)

CUR_WK0 = monday - timedelta(days=17)
CUR_WK = str(CUR_WK0.year) + str(CUR_WK0.month).zfill(2) + str(CUR_WK0.day).zfill(2)

PRE_WK0 = monday - timedelta(days=24)
PRE_WK = PRE_WK0.strftime("%Y%m%d")

MONTH_OFF = CUR_WK0.replace(day=1).date()

In [3]:
# Grabbing IW Release File 
s3 = s3fs.S3FileSystem()
bucket_list = s3.listdir(f'vortex-staging-a65ced90/PYADM/raw/{CUR_PROC_WK}/inbound/')
for file in bucket_list:
    if (file['Key'].__contains__(f"IRWD_RELEASE_WKLY_{CUR_PROC_WK}")):
        release_file = file['Key']

# Reading Release File -
s3_client = boto3.client('s3')
bucket = release_file.split('/')[0]
file_key = release_file.split('/',1)[1]

adm = pd.read_csv(f's3://{bucket}/{file_key}',sep='|')
adm1 = adm[CUR_PROC_WK].tolist()

for item in adm1:
    if item.startswith('P_OUT_110_IRWD_PERIOD_'):
        PERIOD_TIMESTAMP = item.split('_')[-1].split('.')[0]
    elif item.startswith('P_OUT_110_IRWD_WKLY_RX_'):
        RX_TIMESTAMP = item.split('_')[-1].split('.')[0]

Library Names and File Paths :

In [4]:
raw_path = f'PYADM/raw/{CUR_PROC_WK}/inbound/'
curwk = f'PYADM/raw/{CUR_PROC_WK}/dataframes/'
prewk = f'PYADM/raw/{PRE_PROC_WK}/dataframes/'
wkxpn = 'PYADM/weekly/staging/xponent/'
mthxpn = 'PYADM/monthly/staging/xponent/'
pwkxpn = f'PYADM/weekly/archive/{PRE_WK}/xponent/'

In [5]:
temp_PERIOD_CUR_WK = pd.read_csv(
    f's3://{bucket}/{raw_path}P_OUT_110_IRWD_PERIOD_{PERIOD_TIMESTAMP}.TXT',
    delimiter='|',parse_dates=[0,1,3,5]
)

curwk_PERIOD_CUR_WK_v0 = temp_PERIOD_CUR_WK.set_index('PERIOD_KEY',verify_integrity = True) #creates index on priod key (unsure why)



In [6]:
# WEEKLY PARAMETER

temp_RX_PERIOD_TIMESORT = (curwk_PERIOD_CUR_WK_v0['WK_END_DATE'].copy()).to_frame().reset_index(drop=True)
temp_RX_PERIOD_TIMESORT.drop_duplicates(subset = ['WK_END_DATE'],inplace = True)
temp_RX_PERIOD_TIMESORT.sort_values(by = 'WK_END_DATE',inplace = True,ignore_index=True)

W = temp_RX_PERIOD_TIMESORT['WK_END_DATE'].max()
data = [{'I':i,'WK_END_DATE': W - pd.DateOffset(weeks=i-1)} for i in range(1, 106)]
curwk_DATE_PARM_WK = pd.DataFrame(data)
#curwk_DATE_PARM_WK['WK_END_DATE'] = curwk_DATE_PARM_WK['WK_END_DATE'].dt.strftime("%Y%m%d")

# in sas we did i-1  + 5 for some reason, added I as a column, convereted WK_END_DATE to str

In [7]:
# MONTHLY PARAMETER 
temp_RX_PERIOD_MTHSORT = (curwk_PERIOD_CUR_WK_v0['CAL_MONTH'].copy()).to_frame().reset_index(drop=True)
temp_RX_PERIOD_MTHSORT.drop_duplicates(subset = ['CAL_MONTH'],inplace = True)
temp_RX_PERIOD_MTHSORT.sort_values(by = 'CAL_MONTH',inplace = True,ignore_index=True)

month_end_date = CUR_WK0.replace(day=1) - timedelta(days=1)
if 0 <= (CUR_WK0 - month_end_date).days < 7: # When its Month Ending (we use MONTH OFF to start subtracting)
    MT = MONTH_OFF - pd.DateOffset(days = 1)
else:                                        # When its not month ending we take max date from CAL_MONTH value from file
    MT = temp_RX_PERIOD_MTHSORT['CAL_MONTH'].max()

data = [{'I':i,'CCYYMM':MT - pd.DateOffset(months=i-1),'DATE_AS_OF':W} for i in range(1,25)]
curwk_DATE_PARM_MTH = pd.DataFrame(data)
#will need to truncate date to show YYYYMM or a string ?
curwk_DATE_PARM_MTH['CCYYMM'] = curwk_DATE_PARM_MTH['CCYYMM'].dt.strftime("%Y%m") #Comment this if not needed
temp_PERIOD_CUR_WK.shape

(180, 8)

In [8]:
# Store Datasets as required
# sdap(curwk,curwk_DATE_PARM_WK)
# sdap(curwk,curwk_DATE_PARM_MTH)

curwk_DATE_PARM_WK.to_parquet(f's3://{bucket}/{curwk}curwk_DATE_PARM_WK.parquet')
curwk_DATE_PARM_MTH.to_parquet(f's3://{bucket}/{curwk}curwk_DATE_PARM_MTH.parquet')

### 5 - RX Delta - input feed

In [9]:
curwk_RAW_RX_CUR_WK = pd.read_csv(
    f's3://{bucket}/{raw_path}P_OUT_110_IRWD_WKLY_RX_{RX_TIMESTAMP}.TXT',
    delimiter='|',dtype={7:str}
)
curwk_RAW_RX_CUR_WK['MKT_CD'] = curwk_RAW_RX_CUR_WK['MarketName'].str.slice(0, 3)

curwk_RAW_RX_CUR_WK.sort_values(by='PeriodKey',ignore_index=True,inplace=True)

In [10]:
# Added this step to trim weekly delta data for restatements older than 105 weeks.
threshold_105 = curwk_DATE_PARM_WK['WK_END_DATE'].min().strftime('%Y%m%d')
curwk_RAW_RX_CUR_WK = curwk_RAW_RX_CUR_WK[curwk_RAW_RX_CUR_WK['PeriodKey'] >= threshold_105]

In [11]:
print("5.1: Check the period key to see if there is any restatement and missing value")
frequency_table = curwk_RAW_RX_CUR_WK.groupby(['PeriodKey']).size().reset_index(name='FREQUENCY')
frequency_table

5.1: Check the period key to see if there is any restatement and missing value


PeriodKey  FREQUENCY
0    20220513         10
1    20220520         12
2    20220527          4
3    20220603          2
4    20220610         17
..        ...        ...
117  20240419         33
118  20240426         18
119  20240430          5
120  20240503          9
121  20240510     224535

[122 rows x 2 columns]

In [12]:
print("5.2: Check the MKT_CD to see if there is new market and missing value")
frequency_table = curwk_RAW_RX_CUR_WK.groupby(['MKT_CD']).size().reset_index(name='FREQUENCY')
frequency_table

5.2: Check the MKT_CD to see if there is new market and missing value


MKT_CD  FREQUENCY
0    LAX     226062

In [13]:
#APPLY MERGE CASES  
#rdap(curwk,'curwk_MERGES')
curwk_MERGES = pd.read_parquet(f's3://{bucket}/{curwk}curwk_MERGES.parquet')
temp_MERGECASES = curwk_MERGES[['IID','IronwoodWinnerID','IronwoodLoserID']].copy()
temp_MERGECASES.drop_duplicates(subset='IID',inplace=True)
temp_MERGECASES.sort_values(by='IID',ignore_index=True,inplace=True)
mask = (temp_MERGECASES['IID'] == temp_MERGECASES['IronwoodWinnerID'])
fdf = temp_MERGECASES[mask]
print("This should always be 0 : ",fdf.shape[0])

This should always be 0 :  0


In [14]:
groups = temp_MERGECASES.groupby('IronwoodWinnerID')
filtered_df = groups.filter(lambda x: x['IronwoodLoserID'].nunique() > 1)
filtered_df.sort_values(by='IronwoodWinnerID',inplace=True,ignore_index=True)
print("5.3: Check If merge case is valid, List of two loser iid are merged to one winner : ")
filtered_df.head()

5.3: Check If merge case is valid, List of two loser iid are merged to one winner : 


IID  IronwoodWinnerID  IronwoodLoserID
0  12818512            278454         12818512
1   1534124            278454          1534124
2   1917104           2524548          1917104
3  13230484           2524548         13230484
4  13524672           2726667         13524672

In [15]:
curwk_period = curwk_PERIOD_CUR_WK_v0.reset_index()
curwk_period = pl.from_pandas(curwk_period)

curwk_period = curwk_period.with_columns(
    pl.col('PERIOD_KEY').dt.date(),
    pl.col('WK_END_DATE').dt.date(),
    pl.col('CAL_MONTH').dt.date()
)

curwk_period = curwk_period.with_columns(pl.col("PERIOD_KEY").dt.strftime("%Y%m%d").alias("PeriodKey"))

curwk_period = curwk_period.with_columns(pl.col("CAL_MONTH").dt.strftime("%Y%m").alias("CCYYMM"))

curwk_period=curwk_period.drop(["PERIOD_KEY","PERIOD_START_DAY","PERIOD_DAYS","WK_NUM","445_MONTH","445_MO_NUM"])
curwk_period.to_pandas().to_parquet(f's3://{bucket}/{curwk}curwk_period.parquet')

In [16]:
#added this to modify dropweek logic , now covers split week also
dw = (curwk_DATE_PARM_WK['WK_END_DATE'].min() - timedelta(days=7)).strftime("%Y-%m-%d")
dw_lookup = curwk_period.filter(pl.col('WK_END_DATE') == (datetime.strptime(dw, "%Y-%m-%d")))['PeriodKey']

##### Make sure your previous week's dataset is trimmed to its respective 105 week's worth of data ,
- Size is a big concern here or cell crashes

In [17]:
# Splitting Historical data into looser vs non loser.

#Reading Historical Data
prewk_RAW_RX_FIN = pl.read_parquet(f's3://{bucket}/{prewk}RAW_RX_FIN.parquet')
# Dropping 1 week form previous week historical data to have 104 weeks.
prewk_RAW_RX_FIN = prewk_RAW_RX_FIN.filter(~pl.col('PeriodKey').is_in(dw_lookup))
temp_MERGECASES_ids  = list(temp_MERGECASES['IID'].unique())
temp_RAW_RX_FIN = pl.DataFrame()

In [18]:
chunksize = 2_000_000
for i in range(0,len(prewk_RAW_RX_FIN),chunksize):
    chunk = prewk_RAW_RX_FIN.slice(i, min(chunksize, len(prewk_RAW_RX_FIN) - i))
    fc = chunk.filter(~pl.col('IronwoodID').is_in(temp_MERGECASES_ids))
    temp_RAW_RX_FIN = temp_RAW_RX_FIN.vstack(fc)
    
temp_MERGECASES = pl.from_pandas(temp_MERGECASES)
temp_MERGECASES = temp_MERGECASES.rename({'IID' : 'IronwoodID'})
temp_MERGECASES2 = prewk_RAW_RX_FIN.join(temp_MERGECASES,how = 'inner',on='IronwoodID')

#QC check 
print(temp_MERGECASES2.shape[0])
print(prewk_RAW_RX_FIN.shape[0]-temp_RAW_RX_FIN.shape[0])

#print(prewk_RAW_RX_FIN.estimated_size() / 1024 / 1024 / 1024)
#6.176180515438318

399
399


In [19]:
# FOR MEMORY Conservation - 
del prewk_RAW_RX_FIN
gc.collect()

21

In [20]:
group_cols = ['IronwoodWinnerID','MarketID','MarketName','MKT_CD','ChannelID','ChannelName','ProductID','ProductName','PeriodKey']
sum_cols = ['TotalRx','TotalQuantity','TotalFactoredQuantity','NewRx','NewQuantity','NewFactoredQuantity']

#perform groupby and calc function 
temp_MERGECASES3 = temp_MERGECASES2.group_by(group_cols).agg([pl.col(c).sum() for c in sum_cols])
temp_MERGECASES3= temp_MERGECASES3.rename({'IronwoodWinnerID':'IronwoodID'}) #this was mergecases4 step in sas

In [21]:

temp_RAW_RX_CUR_WK = pl.from_pandas(curwk_RAW_RX_CUR_WK)
del curwk_RAW_RX_CUR_WK
gc.collect()

#/*QC TO SEE IF CURRENT WEEK XPN HAS LODERID*/
temp_CURRENTWEEK_CHECK = temp_RAW_RX_CUR_WK.join(temp_MERGECASES,how = 'inner',on='IronwoodID')

print("5.3.1: Check to see if current xpn has loser id")
print(len(temp_CURRENTWEEK_CHECK))

5.3.1: Check to see if current xpn has loser id
0


In [22]:
# Cat Data Fix - #need to make this cat type as well to merge with data
temp_RAW_RX_CUR_WK = temp_RAW_RX_CUR_WK.with_columns([
    pl.col('MarketID').cast(pl.Utf8).cast(pl.Categorical),
    pl.col('MarketName').cast(pl.Utf8).cast(pl.Categorical),
    pl.col('MKT_CD').cast(pl.Utf8).cast(pl.Categorical),
    pl.col('ChannelID').cast(pl.Utf8).cast(pl.Categorical),
    pl.col('ChannelName').cast(pl.Utf8).cast(pl.Categorical),
])

In [23]:
# Combining Historical Data without Loosers + Hist Data with loosers + Rx Delta from raw
temp_RAW_RX_FIN1 = pl.DataFrame()
#temp_RAW_RX_FIN temp_MERGECASES3 temp_RAW_RX_CUR_WK

#need to rearrange column sequence to use vstack-
temp_MERGECASES3 = temp_MERGECASES3.select(temp_RAW_RX_FIN.columns)
temp_RAW_RX_CUR_WK = temp_RAW_RX_CUR_WK.select(temp_RAW_RX_FIN.columns)

temp_RAW_RX_FIN1 = temp_RAW_RX_FIN.vstack(temp_MERGECASES3).vstack(temp_RAW_RX_CUR_WK)

print(temp_RAW_RX_FIN1.shape)

(22355393, 15)


In [24]:
del temp_RAW_RX_FIN
del temp_MERGECASES3
del temp_RAW_RX_CUR_WK
gc.collect()

0

In [25]:
# # Cat Data Fix - #add more and fix later - ONE time action , now historical data is already of type cat | only apply fix on Delta
# temp_RAW_RX_FIN1 = temp_RAW_RX_FIN1.with_columns([
#     pl.col('MarketID').cast(pl.Utf8).cast(pl.Categorical),
#     pl.col('MarketName').cast(pl.Utf8).cast(pl.Categorical),
#     pl.col('MKT_CD').cast(pl.Utf8).cast(pl.Categorical),
#     pl.col('ChannelID').cast(pl.Utf8).cast(pl.Categorical),
#     pl.col('ChannelName').cast(pl.Utf8).cast(pl.Categorical),
# ])

In [26]:
# this should be around 3.2 GB , if more kernel might crash | tested at 13 GB available Ram Env
temp_RAW_RX_FIN1.estimated_size(unit='gb')

2.2092536566779017

In [27]:
## This approach Crashed the kernel 

group_cols = ['MarketID','MarketName','MKT_CD','ChannelID','ChannelName','IronwoodID','ProductID','ProductName','PeriodKey']
sum_cols = ['TotalRx','TotalQuantity','TotalFactoredQuantity','NewRx','NewQuantity','NewFactoredQuantity']

#perform groupby and calc function 
curwk_RAW_RX_FIN = pl.DataFrame()
curwk_RAW_RX_FIN = temp_RAW_RX_FIN1.group_by(group_cols).agg([pl.col(c).sum() for c in sum_cols])

In [28]:
# This took 30 Sec to export at 19 Million records , File size of 230 MB 
# FURTHER TESTING FOR write_parquet() arguments like compression type & level is requried

#curwk_RAW_RX_FIN.write_parquet(curwk+'\\RAW_RX_FIN.parquet',compression='zstd',compression_level=10,use_pyarrow=True)

curwk_RAW_RX_FIN.to_pandas().to_parquet(
    f's3://{bucket}/{curwk}RAW_RX_FIN.parquet',compression='snappy'
)

In [29]:
temp_RX_PERIOD_CUR_WK = pl.DataFrame()
temp_RX_PERIOD_CUR_WK = curwk_RAW_RX_FIN.join(curwk_period,how = 'left',on = 'PeriodKey')
check = list(temp_RX_PERIOD_CUR_WK['WK_END_DATE'].unique())
print("This should always be 105 ! ")
print(len(check))
check = temp_RX_PERIOD_CUR_WK.filter(pl.col("WK_END_DATE").is_null())
print("This should always be 0 ! ")
print(len(check))

This should always be 105 ! 
105
This should always be 0 ! 
0


### SUMMARY

In [30]:
group_cols = ['IronwoodID','WK_END_DATE','CCYYMM','MKT_CD','MarketName','ChannelID','ChannelName','ProductID','ProductName']
sum_cols = ['TotalRx','TotalQuantity','TotalFactoredQuantity','NewRx','NewQuantity','NewFactoredQuantity']

# #perform groupby and calc function 
curwk_RX_PERIOD_WKSUM_CUR_WK = pl.DataFrame()
curwk_RX_PERIOD_WKSUM_CUR_WK = temp_RX_PERIOD_CUR_WK.group_by(group_cols).agg([pl.col(c).sum() for c in sum_cols])

del temp_RX_PERIOD_CUR_WK
gc.collect()

0

In [31]:
#/*Check If There Is Any New Product Not Captured In Standard Market Definition*/
std_sec_def = pl.read_parquet(f's3://{bucket}/PYADM/reference/std_sec_def.parquet')
mkt_bin = set(std_sec_def['PRODUCT_NAME'].unique())
prod_list = set(curwk_RX_PERIOD_WKSUM_CUR_WK['ProductName'].unique())
print("5.4: Check If There Is Any New Product Not Captured In Standard Market Definition")
print(mkt_bin-prod_list)

5.4: Check If There Is Any New Product Not Captured In Standard Market Definition
{'HEPTALAC', 'CONSTILAC', 'EVALOSE', 'PEG 3350-GRX', 'CATULAC', 'CHRONULAC', 'FREELAX', 'CALULOSE', 'CHOLAC', 'CEPHULAC', 'DUPHALAC'}


In [32]:
# Exporting to use for other calc-
curwk_RX_PERIOD_WKSUM_CUR_WK.to_pandas().to_parquet(
    f's3://{bucket}/{curwk}RX_PERIOD_WKSUM_CUR_WK.parquet',compression='snappy'
)
# END OF 5th CODE

In [44]:
## QC TESTING FOR 2023-12-08's Run : Data seems to be correct and ending up with 105 weeks worth of data
# but for some reason when i try to sum it up , kernel is crashing now # Needs more looking into

# Notes for future :
# Really need to come up with a way to perform aggrgation and summing for curwk.RAW_RX_FIN in chunks , 
# try exploring sorting algorithms , and then process chunks 
# current approach is a very unreliable method to execute this and WILL NOT SCALE IN THE FUTURE IF number of HCPS start going up
# Even Truncating data to 105 will not suffice indefinately